<a href="https://colab.research.google.com/github/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/advanced_topics/dicom_private_tags_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Searching private DICOM tags using IDC BigQuery index

This notebook gives an introduction into how private tags can be used to search IDC content and select cohorts using SQL queries and IDC BigQuery index. Note that this is an advanced topic, and it might be easier for you to tackle this after completing the ["Getting started" tutorial series](https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/part3_exploring_cohorts.ipynb) that will introduce you to working with IDC using BigQuery index.

To be able to use this notebook, you will first need to complete the BigQuery prerequisites as discussed in another notebook here: https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/part1_prerequisites.ipynb.

This notebook was designed and tested in Google Colab. Please use the "Open in Colab" button at the top of the notebook to open it in Google Colab. You will need to do extra steps to make it work outside of Google Colab. If you need help with that, please open a thread in IDC Forum: https://discourse.canceridc.dev.

---

Initial release: Dec 2023

Updated: Jan 2025

---

Assuming you completed the aforementioned prerequisites, please run the following cell after entering your GCP project ID in the cell below.

In [1]:
#@title Enter your Project ID
# initialize this variable with your Google Cloud Project ID!
my_ProjectID = "idc-sandbox-000" #@param {type:"string"}

import os
os.environ["GCP_PROJECT_ID"] = my_ProjectID

from google.colab import auth
auth.authenticate_user()

## Background

Imaging equipment manufacturers, unfortunately, sometimes rely on private elements to communicate some of the acquisition details. Even more unfortunate is that in some cases, information encoded in such private attributes is absolutely essential for interpretation of those images. As an example, key acquisition parameters, such as b-values in Diffusion Weighted MRI (DWI), are stored in private elements.

What is a "private element"? This is an element that is not part of the DICOM standard, but instead is specific to a particular vendor.

The use of private elements is problematic for a number of reasons:
* users need to rely on other sources of information than the DICOM standard (typically, manufacturer DICOM conformance statements) to learn about those attributes
* the use of the private elements can change across the software versions
* private elements may be removed as part of image de-identification
* the specific tag group is not guaranteed to be fixed

Fortunately for IDC users, private DICOM elements are extracted and are searchable in BigQuery (see documentation here https://cloud.google.com/healthcare-api/docs/how-tos/dicom-bigquery-schema#private_data_elements).

In this notebook we demonstrate how private tags and corresponding data can be accessed using BigQuery on an example of the prostate DWI use case.



## Filtering images by DWI b-values

### Understanding where to find b-values

Private DICOM elements that are not sequences (not DICOM SQ VR) are extracted into the BigQuery column `OtherElements` in the IDC BigQuery index (`bigquery-public-data.idc_current.dicom_all`).

First thing to remember is that since private elements are not harmonized across vendors, you will first need to identify the subset of images that you want to analyze, identify the manufacturer and model name for the imaging equipment that produced this specific data, locate DICOM conformance statement and understand how the values of interest are encoded.

For the sake of this example, we will look into DWI images that are distributed as part of the [QIN-Prostate-Repeatability collection](http://doi.org/10.7937/K9/TCIA.2018.MR1CKGND). B-value is a key acquisition parameter for DWI.

Let's first get an idea about the manufacturer and model name for the MR images included in this collection.

In [2]:
%%bigquery --project $my_ProjectID
SELECT
  DISTINCT Manufacturer, ManufacturerModelName
FROM
  `bigquery-public-data.idc_current.dicom_all`
WHERE
  collection_id = "qin_prostate_repeatability" AND Modality = "MR"

Query is running:   0%|          |

Downloading:   0%|          |

,Manufacturer,ManufacturerModelName
0,GE MEDICAL SYSTEMS,DISCOVERY MR750w
1,GE MEDICAL SYSTEMS,Signa HDxt


With a little help from ChatGPT (you will have to use your preferred search engine, as there is no single resource containing all of the DICOM conformance statements from all vendors), we located the DICOM conformance statement from GE titled ["HDxt, Hdi & Signa Vibrant Product Line DICOM CONFORMANCE STATEMENT"](https://www.gehealthcare.com/-/jssmedia/widen/2018/01/25/0204/gehealthcarecom/migrated/2018/02/19/0841/tic-resonance-gehc-dicom-conformance_signa-hdi-hdxt-15-30-vibrant-sw16_doc0878555_rev2_pdf.pdf?hash=2F2C737642001B6425A26406F04E614F&rev=-1).

This is the page that contains a rather long list of conformance statements from GE: https://www.gehealthcare.com/products/interoperability/dicom/magnetic-resonance-imaging-dicom-conformance-statements, which also includes one for Discovery 750w.

Opening the PDF documents for those statements, we can search for text string `b_value`, and locate the table that indicates that private element `(0043,1039)` is a sequence (array), which contains b-value as the first item in that sequence.

### Accessing specific private elements in the search query

Now that we know that b-values are in the first item of the `(0043,1039)` sequence, we can access its values.

In [3]:
%%bigquery --project $my_ProjectID

 SELECT
    collection_id,
    StudyDate,
    StudyInstanceUID,
    SeriesInstanceUID,
    SeriesDescription,
    SOPInstanceUID,
    other_elements.Data[SAFE_OFFSET(0)] AS b_value
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND other_elements.Tag = "Tag_00431039"

Query is running:   0%|          |

Downloading:   0%|          |

,collection_id,StudyDate,StudyInstanceUID,SeriesInstanceUID,SeriesDescription,SOPInstanceUID,b_value
0,qin_prostate_repeatability,1995-12-23,1.3.6.1.4.1.14519.5.2.1.3671.4754.264231410930...,1.3.6.1.4.1.14519.5.2.1.3671.4754.326093319837...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.283419705608...,0
1,qin_prostate_repeatability,1994-07-02,1.3.6.1.4.1.14519.5.2.1.3671.4754.288848219213...,1.3.6.1.4.1.14519.5.2.1.3671.4754.768464301267...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.116999121592...,0
2,qin_prostate_repeatability,1995-12-23,1.3.6.1.4.1.14519.5.2.1.3671.4754.264231410930...,1.3.6.1.4.1.14519.5.2.1.3671.4754.326093319837...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.318336603973...,0
3,qin_prostate_repeatability,1996-03-24,1.3.6.1.4.1.14519.5.2.1.3671.4754.194890890650...,1.3.6.1.4.1.14519.5.2.1.3671.4754.172888993627...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.261878383043...,0
4,qin_prostate_repeatability,1996-05-02,1.3.6.1.4.1.14519.5.2.1.3671.4754.182983245842...,1.3.6.1.4.1.14519.5.2.1.3671.4754.120594212806...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.224027194726...,0
...,...,...,...,...,...,...,...
37949,qin_prostate_repeatability,1995-12-15,1.3.6.1.4.1.14519.5.2.1.3671.4754.298665348758...,1.3.6.1.4.1.14519.5.2.1.3671.4754.223103497359...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.145358042372...,0
37950,qin_prostate_repeatability,1996-05-16,1.3.6.1.4.1.14519.5.2.1.3671.4754.431678793261...,1.3.6.1.4.1.14519.5.2.1.3671.4754.628634179380...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.769744734224...,0
37951,qin_prostate_repeatability,1995-08-26,1.3.6.1.4.1.14519.5.2.1.3671.4754.131608903441...,1.3.6.1.4.1.14519.5.2.1.3671.4754.714877156889...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.782973591194...,0
37952,qin_prostate_repeatability,1994-09-03,1.3.6.1.4.1.14519.5.2.1.3671.4754.394942238206...,1.3.6.1.4.1.14519.5.2.1.3671.4754.242726537614...,DCE,1.3.6.1.4.1.14519.5.2.1.3671.4754.230736230817...,0


As you can see, we are getting a lot of images that have b-value 0. Remember, that each row in the IDC BigQuery index corresponds to a single DICOM file, which for most MR images corresponds to a single slice.

Let's next create a summary table that lists all b-values encountered for every series in our collection.

In [4]:
%%bigquery --project $my_ProjectID

 SELECT
    SeriesInstanceUID,
    ANY_VALUE(SeriesDescription) as SeriesDescription,
    ARRAY_AGG(DISTINCT(other_elements.Data[SAFE_OFFSET(0)])) as b_values
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND Modality = "MR"
    AND other_elements.Tag = "Tag_00431039"
  GROUP BY
    SeriesInstanceUID

Query is running:   0%|          |

Downloading:   0%|          |

,SeriesInstanceUID,SeriesDescription,b_values
0,1.3.6.1.4.1.14519.5.2.1.3671.4754.338695413278...,DCE Subtraction,[0]
1,1.3.6.1.4.1.14519.5.2.1.3671.4754.325025234684...,T2 Weighted Axial,[0]
2,1.3.6.1.4.1.14519.5.2.1.3671.4754.319423864233...,DCE,[0]
3,1.3.6.1.4.1.14519.5.2.1.3671.4754.423145802782...,DWI,"[0, 1400]"
4,1.3.6.1.4.1.14519.5.2.1.3671.4754.983460207615...,T2 Weighted Axial,[0]
...,...,...,...
129,1.3.6.1.4.1.14519.5.2.1.3671.4754.253470466967...,DCE Subtraction,[0]
130,1.3.6.1.4.1.14519.5.2.1.3671.4754.165941479363...,T2 Weighted Axial,[0]
131,1.3.6.1.4.1.14519.5.2.1.3671.4754.864172514337...,DCE,[0]
132,1.3.6.1.4.1.14519.5.2.1.3671.4754.204853722013...,DWI,"[0, 1400]"


We have 134 MR series that have b-value initialized. Each of those series belongs to one of the following three groups:

1. b-value = [0]
2. b-value = [1400]
3. b-value = [0,1400]

We are interested in high-b DWI images, but you can also see that "Apparent Diffusion Coefficient" series has b-value of 1400, but it is not the DWI acquisition, but a parameter map derived from DWI.

When you do this kind of explorations, it is always helpful to visualize individual series so that you can confirm they are what you think they are, and also look at the DICOM tags. Let's modify the query slightly to add the URL that you can use to open each series in IDC viewer. This time we will use Python so that we can configure the output cell to hyperlink the URLs.



In [5]:
from IPython.display import HTML
from google.cloud import bigquery

# BigQuery client is initialized with the ID of the project
# we specified in the beginning of the notebook!
bq_client = bigquery.Client(my_ProjectID)

selection_query = """
 SELECT
    SeriesInstanceUID,
    ANY_VALUE(SeriesDescription) as SeriesDescription,
    ARRAY_AGG(DISTINCT(other_elements.Data[SAFE_OFFSET(0)])) as b_values,
    ANY_VALUE(CONCAT('https://viewer.imaging.datacommons.cancer.gov/viewer/',
     StudyInstanceUID,
     '?SeriesInstanceUID=',
     SeriesInstanceUID)) as ohif_v2_url,
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND Modality = "MR"
    AND other_elements.Tag = "Tag_00431039"
  GROUP BY
    SeriesInstanceUID
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

def make_clickable(val):
    return '<a href="{}" target="_blank">{}</a>'.format(val,val)

selection_df['ohif_v2_url'] = selection_df['ohif_v2_url'].apply(make_clickable)

# Display the DataFrame with clickable URLs
HTML(selection_df.to_html(escape=False))

,SeriesInstanceUID,SeriesDescription,b_values,ohif_v2_url
0,1.3.6.1.4.1.14519.5.2.1.3671.4754.306182248785281519091172718544,Apparent Diffusion Coefficient,[1400],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.304001015100152820571364796240?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.306182248785281519091172718544
1,1.3.6.1.4.1.14519.5.2.1.3671.4754.215087918941937902339190030623,DWI,"[0, 1400]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.162322077241419864654545266845?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.215087918941937902339190030623
2,1.3.6.1.4.1.14519.5.2.1.3671.4754.301965004788702079527239108078,DWI,"[0, 1400]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.217251077299095981018478951589?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.301965004788702079527239108078
3,1.3.6.1.4.1.14519.5.2.1.3671.4754.161100404117807686389503159717,DCE,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.162322077241419864654545266845?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.161100404117807686389503159717
4,1.3.6.1.4.1.14519.5.2.1.3671.4754.654327272206188613770270554127,T2 Weighted Axial,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.654327272206188613770270554127
5,1.3.6.1.4.1.14519.5.2.1.3671.4754.289381776301402611413405156513,T2 Weighted Axial,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.173585825705285812758858023242?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.289381776301402611413405156513
6,1.3.6.1.4.1.14519.5.2.1.3671.4754.347758533160754099807506213512,DCE Subtraction,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.962865348146140991373362412648?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.347758533160754099807506213512
7,1.3.6.1.4.1.14519.5.2.1.3671.4754.328402048123080895584744371789,DWI,"[1400, 0]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.240243858523310211919840278205?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.328402048123080895584744371789
8,1.3.6.1.4.1.14519.5.2.1.3671.4754.228976611072825280871903738095,DCE,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.568383070573576760578239627478?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.228976611072825280871903738095
9,1.3.6.1.4.1.14519.5.2.1.3671.4754.199355443956588961060720091494,Apparent Diffusion Coefficient,[1400],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.263464522348816475234793721892?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.199355443956588961060720091494


After opening the series in the viewer, you can open "Tag Browser" to see all of the DICOM attributes, confirm your assumptions and understand the data better.

In order to filter out ADC series, we can use another - this time, standard! - DICOM attribute `ImageType`. You can see that for DWI series, the value of this attribute is `ORIGINAL\PRIMARY\OTHER`, while for ADC (since this is a derived image series), it is `DERIVED\SECONDARY\PROCESSED`.

In [6]:
selection_query = """
 SELECT
    SeriesInstanceUID,
    ANY_VALUE(SeriesDescription) as SeriesDescription,
    ANY_VALUE(ImageType[SAFE_OFFSET(0)]) as original_or_not,
    ARRAY_AGG(DISTINCT(other_elements.Data[SAFE_OFFSET(0)])) as b_values,
    ANY_VALUE(CONCAT('https://viewer.imaging.datacommons.cancer.gov/viewer/',
     StudyInstanceUID,
     '?SeriesInstanceUID=',
     SeriesInstanceUID)) as ohif_v2_url,
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND Modality = "MR"
    AND other_elements.Tag = "Tag_00431039"
  GROUP BY
    SeriesInstanceUID
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

def make_clickable(val):
    return '<a href="{}" target="_blank">{}</a>'.format(val,val)

selection_df['ohif_v2_url'] = selection_df['ohif_v2_url'].apply(make_clickable)

# Display the DataFrame with clickable URLs
HTML(selection_df.to_html(escape=False))

,SeriesInstanceUID,SeriesDescription,original_or_not,b_values,ohif_v2_url
0,1.3.6.1.4.1.14519.5.2.1.3671.4754.190547402621364290155124030251,DWI,ORIGINAL,"[1400, 0]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.182983245842036631781777324507?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.190547402621364290155124030251
1,1.3.6.1.4.1.14519.5.2.1.3671.4754.255454967755971688897901580729,DWI,ORIGINAL,"[0, 1400]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.568383070573576760578239627478?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.255454967755971688897901580729
2,1.3.6.1.4.1.14519.5.2.1.3671.4754.181371867300403251927910063803,DCE Subtraction,DERIVED,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.288453744990673175783381845880?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.181371867300403251927910063803
3,1.3.6.1.4.1.14519.5.2.1.3671.4754.189038321516128072700572876674,DCE Subtraction,DERIVED,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.117496148504386222592571043379?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.189038321516128072700572876674
4,1.3.6.1.4.1.14519.5.2.1.3671.4754.228015946741563785297552112143,DCE,ORIGINAL,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.233298968565250099224990877587?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.228015946741563785297552112143
5,1.3.6.1.4.1.14519.5.2.1.3671.4754.882539338600550863063890292531,Apparent Diffusion Coefficient,DERIVED,[1400],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.133806669697115873569905534398?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.882539338600550863063890292531
6,1.3.6.1.4.1.14519.5.2.1.3671.4754.340967763797966622706944397525,DWI,ORIGINAL,"[0, 1400]",https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.121472087445374646718121301133?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.340967763797966622706944397525
7,1.3.6.1.4.1.14519.5.2.1.3671.4754.263209796341977244509548411569,DCE,ORIGINAL,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.204078048920967920419177848890?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.263209796341977244509548411569
8,1.3.6.1.4.1.14519.5.2.1.3671.4754.114951445141329894346883632256,DCE Subtraction,DERIVED,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.162322077241419864654545266845?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.114951445141329894346883632256
9,1.3.6.1.4.1.14519.5.2.1.3671.4754.322350301601580056579486694857,T2 Weighted Axial,ORIGINAL,[0],https://viewer.imaging.datacommons.cancer.gov/viewer/1.3.6.1.4.1.14519.5.2.1.3671.4754.568383070573576760578239627478?SeriesInstanceUID=1.3.6.1.4.1.14519.5.2.1.3671.4754.322350301601580056579486694857


Now we can finally define the selection criteria for high-b DWI images. They should:
* have first element of `ImageType` set to "ORIGINAL"
* have b-value stored in the first element of private tag `(0043,1039)` set to "1400"

The query below will select series that meet these criteria, and count the slices.

In [11]:
selection_query = """
 SELECT
    ANY_VALUE(PatientID) as PatientID,
    ANY_VALUE(StudyDate) as StudyDate,
    SeriesInstanceUID,
    ANY_VALUE(SeriesDescription) as SeriesDescription,
    ARRAY_AGG(DISTINCT(other_elements.Data[SAFE_OFFSET(0)])) as b_values,
    COUNT(DISTINCT(SOPInstanceUID)) as n_slices
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND Modality = "MR"
    AND other_elements.Tag = "Tag_00431039"
    AND ImageType[SAFE_OFFSET(0)] = "ORIGINAL"
    AND other_elements.Data[SAFE_OFFSET(0)] = "1400"
  GROUP BY
    SeriesInstanceUID
  ORDER BY
    PatientID,
    StudyDate
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()

# Display the DataFrame with clickable URLs
HTML(selection_df.to_html(escape=False))

,PatientID,StudyDate,SeriesInstanceUID,SeriesDescription,b_values,n_slices
0,PCAMPMRI-00001,1994-06-20,1.3.6.1.4.1.14519.5.2.1.3671.4754.340967763797966622706944397525,DWI,[1400],20
1,PCAMPMRI-00001,1994-07-02,1.3.6.1.4.1.14519.5.2.1.3671.4754.317298100327363935801567299386,DWI,[1400],20
2,PCAMPMRI-00002,1994-07-09,1.3.6.1.4.1.14519.5.2.1.3671.4754.334381723390926827137962689915,DWI,[1400],20
3,PCAMPMRI-00002,1994-07-23,1.3.6.1.4.1.14519.5.2.1.3671.4754.255454967755971688897901580729,DWI,[1400],16
4,PCAMPMRI-00003,1994-08-11,1.3.6.1.4.1.14519.5.2.1.3671.4754.215087918941937902339190030623,DWI,[1400],20
5,PCAMPMRI-00003,1994-08-25,1.3.6.1.4.1.14519.5.2.1.3671.4754.328198183251927452263461016021,DWI,[1400],16
6,PCAMPMRI-00004,1994-09-03,1.3.6.1.4.1.14519.5.2.1.3671.4754.231419317210078729036516968070,DWI,[1400],16
7,PCAMPMRI-00004,1994-09-12,1.3.6.1.4.1.14519.5.2.1.3671.4754.369908310009091852940905931770,DWI,[1400],16
8,PCAMPMRI-00005,1994-09-10,1.3.6.1.4.1.14519.5.2.1.3671.4754.337665444208782319977763709123,DWI,[1400],16
9,PCAMPMRI-00005,1994-09-17,1.3.6.1.4.1.14519.5.2.1.3671.4754.113884294763670896474464289534,DWI,[1400],24



### Filtering by specific b-value and downloading images

Now that we know how to identify DWI images with multiple b-values, we can filter images that correspond to the high b-value (this is often the image that is used by AI PCa detection tools).

In the query below, we get AWS URLs for all slices that have b-value of 1400. For convenience, we use `PatientID` and `SeriesInstanceUID` attributes to sort the files on the fly as we download them.

The query selects the specific series - you can remove the corresponding check from the query to select all series!

In [25]:
selection_query = """
WITH b_values_added AS (
 SELECT
    CONCAT("cp ",aws_url," ./",PatientID,"/",SeriesInstanceUID,"/") as aws_cp_command,
    other_elements.Data[SAFE_OFFSET(0)] AS b_value
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    collection_id = "qin_prostate_repeatability"
    AND other_elements.Tag = "Tag_00431039"
    AND other_elements.Data[SAFE_OFFSET(0)] = "1400"
    AND ImageType[SAFE_OFFSET(0)] = "ORIGINAL"
    # remove the next line to select all high-b images!
    AND SeriesInstanceUID = "1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789"
  ORDER BY
    aws_cp_command
)

SELECT
  aws_cp_command
FROM
  b_values_added
"""

selection_result = bq_client.query(selection_query)
selection_df = selection_result.result().to_dataframe()


#selection_df.head()# save to csv
with open("aws_manifest.s5cmd", "w") as f:
    for command in selection_df['aws_cp_command']:
        f.write(f"{command}\n")


Now that we have the manifest, we can install [`idc-index`](https://github.com/ImagingDataCommons/idc-index) Python package, and download the corresponding files.

In [8]:
!pip install --upgrade idc-index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.3
    Uninstalling duckdb-1.1.3:
      Successfully uninstalled duckdb-1.1.3
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.1.4 which is incompatible.
plotnine 0.14.5 requires pandas>=2.2.0, but you have pandas 2.1.4 which is incompatible.
mizani 0.13.1 requires pan

In [26]:
from idc_index import IDCClient

c=IDCClient()

!rm -rf dwi_images
!mkdir -p dwi_images
!cd dwi_images && $c.s5cmdPath --no-sign-request --endpoint-url https://s3.amazonaws.com run ../aws_manifest.s5cmd

cp s3://idc-open-data/292d229b-c501-4b3d-8df7-c8615d13df20/64ebd0b9-d0b2-4808-89de-c8e94092e746.dcm PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789/64ebd0b9-d0b2-4808-89de-c8e94092e746.dcm
cp s3://idc-open-data/292d229b-c501-4b3d-8df7-c8615d13df20/d738d690-910f-42d8-b23e-ea9b688f1bac.dcm PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789/d738d690-910f-42d8-b23e-ea9b688f1bac.dcm
cp s3://idc-open-data/292d229b-c501-4b3d-8df7-c8615d13df20/e55d30e0-33b1-4dea-b95d-1ae6764e2471.dcm PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789/e55d30e0-33b1-4dea-b95d-1ae6764e2471.dcm
cp s3://idc-open-data/292d229b-c501-4b3d-8df7-c8615d13df20/1c82df83-9131-459b-92da-416c99ee55be.dcm PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789/1c82df83-9131-459b-92da-416c99ee55be.dcm
cp s3://idc-open-data/292d229b-c501-4b3d-8df7-c8615d13df20/fd0acd8d-441e-44e1-bf0d-19c81a63093b.dcm PCAMPMRI-00009/1

### Working with the downloaded images

In the previous section, we downloaded high-b DWI images and organized them by series.

Typically, AI tools expect input in formats such as NIfTI or NRRD, and this can be accomplished easily by converting from DICOM. For DICOM to NIfTI conversion, we recommend `dcm2niix`, which is very easy to install.

In [27]:
%%capture
!apt-get install dcm2niix

In the next cell we convert one of the high-b DWI images into NIfTI.

In [28]:
!rm -rf nifti_converted
!mkdir -p nifti_converted
!dcm2niix -o ./nifti_converted /content/dwi_images/PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789

Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 24 DICOM file(s)
DICOM images may be missing, expected 48 spatial locations per volume, but found 24 slices.
GE DTI directions require head first supine acquisition
Convert 24 DICOM as ./nifti_converted/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789_Prostate_With_e-coil_C-_19950701142721_8 (256x256x24x1)
Conversion required 0.031092 seconds (0.028232 for core code).


## Visualization of the downloaded images

WARNING: `itkWidgets` is a large package with many dependencies. If you run into problem with this cell, continue to the next section!

Visualization of the high-b DWI image can also be done right here in the notebook using `itkWidgets`, which can read image directly from DICOM. See more details in this tutorial: https://github.com/ImagingDataCommons/IDC-Tutorials/blob/master/notebooks/getting_started/part3_exploring_cohorts.ipynb.

Installing `itkWidgets` will take some time!

In [29]:
!pip install -q "itk-io>=5.3.0" "itkwidgets[all]>=1.0a32"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.9/179.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━

In [30]:
import itk
from itkwidgets import view

image = itk.imread("/content/dwi_images/PCAMPMRI-00009/1.3.6.1.4.1.14519.5.2.1.3671.4754.234511234806445476378897256789")

viewer = view(image, ui_collapsed=False)


viewer.set_background_color([0.1,0.1,0.1])
viewer.set_image_color_map("Grayscale")
viewer.set_image_color_range([0,5000])
viewer.set_label_image_blend(0.75)
viewer.set_view_mode('ZPlane')

AttributeError: module 'zarr.storage' has no attribute 'BaseStore'

## Looking beyond QIN-Prostate-Repeatability

The query below will summarize all of the collections that have the notorious GE-specific b-values tag. As you can see, some of the values appear to follow different conventions, such as 1000000600, which cannot be realistic.

In [ ]:
%%bigquery --project $my_ProjectID

WITH
  b_value_added AS (
  SELECT
    collection_id,
    Manufacturer,
    ManufacturerModelName,
    ARRAY_TO_STRING(SoftwareVersions,"/"),
    PatientID,
    StudyDate,
    StudyInstanceUID,
    SeriesInstanceUID,
    SeriesDescription,
    SOPInstanceUID,
    other_elements.Data[SAFE_OFFSET(0)] AS b_value
  FROM
    `bigquery-public-data.idc_current.dicom_all`,
    UNNEST(OtherElements) AS other_elements
  WHERE
    other_elements.Tag = "Tag_00431039" )
SELECT
  collection_id,
  array_to_string(ARRAY_AGG(DISTINCT(b_value)),",") as b_values_encountered,
  ARRAY_AGG(DISTINCT(CONCAT(SAFE_CAST(Manufacturer AS STRING)," ",ManufacturerModelName)) IGNORE NULLS) as Manufacturer_and_ModelName
FROM
  b_value_added
WHERE
  b_value <> ""
GROUP BY
  collection_id

Query is running:   0%|          |

Downloading:   0%|          |

,collection_id,b_values_encountered,Manufacturer_and_ModelName
0,tcga_prad,"0,600","[GE MEDICAL SYSTEMS GENESIS_SIGNA, GE MEDICAL ..."
1,breast_mri_nact_pilot,"0,450,600",[GE MEDICAL SYSTEMS GENESIS_SIGNA]
2,tcga_lihc,"0,500","[GE MEDICAL SYSTEMS GENESIS_SIGNA, GE MEDICAL ..."
3,cmb_mml,0,"[GE MEDICAL SYSTEMS Signa HDxt, GE MEDICAL SYS..."
4,tcga_brca,"0,1000","[GE MEDICAL SYSTEMS SIGNA HDx, GE MEDICAL SYST..."
5,acrin_6698,"800,1000000800,100,0,1000000600,1000000100,600","[GE MEDICAL SYSTEMS Signa HDxt, GE MEDICAL SYS..."
6,tcga_kirc,"500,1000,0","[GE MEDICAL SYSTEMS SIGNA HDx, GE MEDICAL SYST..."
7,tcga_blca,"0,800","[GE MEDICAL SYSTEMS Signa HDxt, GE MEDICAL SYS..."
8,tcga_cesc,"800,0,100","[GE MEDICAL SYSTEMS SIGNA EXCITE, GE MEDICAL S..."
9,pseudo_phi_dicom_data,0,[GE MEDICAL SYSTEMS GENESIS_SIGNA]


# Next steps

Share your feedback or ask questions about this tutorial in IDC Forum: https://discourse.canceridc.dev.